# Travel Routing Example

In [ ]:
import numpy
from concrete import fhe

## Setting up Server Data

The server owns map data.
The routes network is represented by its weighted adjacency matrix W: $W_{i, j}$ is the distance between nodes $i$ and $j$ if there is an edge between them, and infinity otherwise.

In [ ]:
N_BITS = 6
inf = 2**N_BITS - 1  # infinity
# Origin x Destination -> Weight of edge (infinite if no edge)
weights = numpy.array([
    [  0,   1,   2, inf, inf, inf, inf, inf, inf, ],
    [  1,   0, inf,   1,   1, inf, inf, inf, inf, ],
    [  2, inf,   0, inf, inf,   1,   1, inf, inf, ],
    [inf,   1, inf,   0, inf,   1, inf, inf, inf, ],
    [inf,   1, inf, inf,   0,   1, inf,   1, inf, ],
    [inf, inf,   1,   1,   1,   0, inf, inf,   2, ],
    [inf, inf,   1, inf, inf, inf,   0, inf,   1, ],
    [inf, inf, inf, inf,   1, inf, inf,   0,   1, ],
    [inf, inf, inf, inf, inf,   2,   1,   1,   0, ],
])
N_NODES = weights.shape[0]
assert weights.shape[0] == weights.shape[1]
assert (weights == weights.T).all()
assert (weights.diagonal() == 0).all()

The server than pre-computes shortest paths between all OD pairs (origin and destination). The routing information is stored in a matrix M: $M_{i,j}$ is the next node to visit on the shortest path from node $i$ to node $j$.

In [ ]:
from dijkstra import Dijkstra
router = Dijkstra(weights)
# Matrix of origin x destination -> next node on shortest path
next_nodes = router.get_all_shortest_paths()[:, :, 1]

Private Travel Routing than consist of oblivious transfer of an element of $M$. This is done in Zama via a TableLookup of the flatten matrix $M$.

In [ ]:
routes = fhe.LookupTable(next_nodes.flatten())

@fhe.compiler({"origin": "encrypted", "destination": "encrypted"})
def route(origin, destination):
    return routes[N_NODES * origin + destination]


circuit = route.compile([(0, N_NODES - 1), (N_NODES - 1, 0)])

## Client

The client can stream the routing information privately by requesting the next node iteratively. For maximum privacy, the client can keep asking for next nodes even once the full shortest path is retrieved to hide information about the length of the route. 

In [ ]:
circuit.client.keys.generate()


def shortest_path(origin, destination):
    path = [origin, ]
    o, d = circuit.encrypt(origin, destination)
    for _ in range(N_NODES - 1):
        # Careful: breaking early could lead to information leak
        # if origin == destination:
        #     break
        o = circuit.run(o, d)
        origin = circuit.decrypt(o)
        path.append(origin)
    return path

## Benchmarks

In [ ]:
%%timeit
shortest_path(0, 7)